In [223]:
import numpy as np
import random
import pandas as pd
import itertools
import math
import time
import concurrent.futures

In [224]:
class Coin:
    def __init__(self, bias=0.5):
        self.bias = bias
    def toss(self, n=1000):
        return np.random.binomial(n, self.bias, size=None)

In [275]:
class Agent:
    def __init__(self, no, coinA, coinB, w, k):
        self.no = no
        self.w = w # intergroup distrust
        self.k = k # conformity
        self.coinA = coinA
        self.coinB = coinB
        """Credence is modelled by beta distribution. Record parameters alpha and beta for each coin."""
        self.cred = {self.coinA: np.random.uniform(low=1, high=4, size=2),
                     self.coinB: np.random.uniform(low=1, high=4, size=2)}

    def update(self, coin, data):
        """data in the form [head, tail]"""
        self.cred[coin] = np.sum([self.cred[coin], data], axis=0)
    def exputil(self, alpha, beta, n_in, n_all):
        """n_in: number of in-group members choosing that coin"""
        """n_all: total number of members in network"""
        p = alpha/(alpha + beta)
        return ((1-self.k)*p + self.k*n_in/n_all)
    def choose(self, n_inA, n_inB, n_all):
        """choose which coin to flip based on credence and peer influence"""
        euA = self.exputil(self.cred[self.coinA][0], self.cred[self.coinA][1], n_inA, n_all)
        euB = self.exputil(self.cred[self.coinB][0], self.cred[self.coinB][1], n_inB, n_all)
        print('A', self.cred[self.coinA][0], self.cred[self.coinA][1], euA)
        print('B', self.cred[self.coinB][0], self.cred[self.coinB][1], euB)
        if euA > euB:
            return self.coinA
        else:
            return self.coinB

In [276]:
class Model:
    def __init__(self, w, k, epsilon=0.001, div=0.5, n_all=10):
        np.random.seed()
        self.w = w # intergroup distrust
        self.k = k # conformity
        self.epsilon = epsilon # difference between coins (difficulty)
        self.div = div # subgroup size
        self.n_all = n_all # total number of agents
        self.subgroups = dict()
        """create coins"""
        self.coinA = Coin()
        self.coinB = Coin(0.5+epsilon)
        """create agents"""
        self.agents = []
        for i in range(n_all):
            self.agents.append(Agent(i, self.coinA, self.coinB, self.w, self.k))
        """divide agents into subgroups"""
        size = round(10*self.div)
        self.subgroups = dict()
        for a in self.agents[0:size]:
            self.subgroups[a] = self.agents[0:size]
        for a in self.agents[size:10]:
            self.subgroups[a] = self.agents[size:10]
        """record choices of which coin to flip"""
        self.choices= dict()
        for a in self.agents:
            self.choices[a] = a.choose(0, 0, self.n_all) # no peer influence at the start
        
    def update(self, n_flips=1000):
        """agents flip coin one by one"""
        for a in self.agents:
            coin = self.choices[a]
            result = coin.toss(n=n_flips) # 1000 flips per time step
            data = np.array([result, n_flips-result])
            """each time an agent flips, they share data with everyone (complete network)"""
            for b in self.agents:
                if b in self.subgroups[a]:
                    b.update(coin, data) # update on ingroup data (including own data)
                else:
                    b.update(coin, data*self.w) # update on outgroup data (reduced trust)
                    
    def choose(self):
        """all agents choose which coin to flip in the next round"""
        n_in = {key: [] for key in self.agents}
        for a in self.agents:
            if n_in[a] == []:
                n_inA = sum([int(self.choices[b].bias==0.5) for b in self.subgroups[a]])
                n_inB = len(self.subgroups[a]) - n_inA
                for b in self.subgroups[a]:
                    n_in[b] = (n_inA, n_inB)
        for a in self.agents:
            print(a.no)
            print(n_in[a][0], n_in[a][1], self.n_all)
            self.choices[a] = a.choose(n_in[a][0], n_in[a][1], self.n_all)
            

In [280]:
M = Model(1, 0.02, epsilon=0.001)

A 1.7196579657813573 2.2137427637847766 0.42844981285484735
B 3.030917336703847 1.4371708248814674 0.664780747951017
A 3.9696995666824972 2.425441454430084 0.6083220936810616
B 2.3938354781350073 3.2534301460278674 0.4154149857117906
A 3.0236360273741463 2.844494672882409 0.5049586415478293
B 2.4659187505459608 1.410798151964185 0.6233626123099962
A 1.1831430878233904 1.621647793148305 0.4133927537817831
B 3.4936734419852358 1.5387196684576592 0.6803522495173686
A 1.5230871656413023 2.1694630770737833 0.40422616463329897
B 3.9303880179874717 2.8210295691844003 0.570514296870979
A 1.9107917616380221 1.516764446080851 0.5463297500966466
B 2.367084206844134 1.0738363287824404 0.6741633521289203
A 1.0523322792546277 2.8636435707376062 0.26335342023919284
B 3.2018545407368735 1.3601324914719681 0.6878181432275695
A 3.616751291687527 3.017891592187365 0.5342286431826302
B 1.6221046874167722 1.7677896608698123 0.46894163367419656
A 3.490021878806495 3.981485536094063 0.45776859357850036
B 1.7

In [281]:
[c.bias for c in M.choices.values()]

[0.501, 0.5, 0.501, 0.501, 0.501, 0.501, 0.501, 0.5, 0.5, 0.5]

In [283]:
M.update()
M.choose()
[c.bias for c in M.choices.values()]

0
0 5 10
A 4510.719657965781 4493.2137427637845 0.4909526834625724
B 5441.030917336704 5563.437170824882 0.4945495717077201
1
0 5 10
A 4512.969699566683 4493.42544145443 0.49106332070990155
B 5440.393835478135 5565.253430146027 0.49444092656155125
2
0 5 10
A 4512.023636027374 4493.844494672882 0.4909891083385216
B 5440.465918750546 5563.410798151965 0.4945252939071774
3
0 5 10
A 4510.183143087823 4492.621647793148 0.49095582797742177
B 5441.493673441985 5563.538719668458 0.49456593397322396
4
0 5 10
A 4510.523087165641 4493.1694630770735 0.490944420942402
B 5441.930388017988 5564.821029569184 0.49452913833741446
5
5 0 10
A 4510.910791761638 4492.516764446081 0.5010010713507
B 5440.367084206844 5563.073836328782 0.48453568124971375
6
5 0 10
A 4510.052332279254 4493.863643570738 0.5008810008321322
B 5441.201854540737 5563.3601324914725 0.48456065981850105
7
5 0 10
A 4512.616751291687 4494.017891592188 0.5010118586591003
B 5439.622104687417 5563.767789660869 0.48447157773912597
8
5 0 10
A

[0.501, 0.501, 0.501, 0.501, 0.501, 0.5, 0.5, 0.5, 0.5, 0.5]

In [ ]:
W = [0.005, 0.0125, 0.1, 0.5, 1]
K = [0, 0.0125, 0.025, 0.05, 0.1]

In [13]:
def run_distrust():
    cols = ['k', 'w', 'epsilon', 'n_all', 'div', 
            'alphaA', 'betaA', 'alphaB', 'betaB', 'choices']
    df = pd.DataFrame(columns=cols)
    k = 0 # No conformity
    epsilon = 0.001 # difference between coin A and coin B biases
    n_all = 10 # agents per group
    for w in W: # levels of intergroup trust
        for div in [0.5, 0.7, 0.9]: # subgroup compositions 5-5, 7-3, 9-1
            M = Model(w=w, k=k, epsilon=epsilon, div=div, n_all=n_all)
            for t in range(3000): # 3000 time steps
                M.update()
                M.choose()
            df = df.append(pd.DataFrame([[k, 
                                          w,
                                          epsilon,
                                          n_all,
                                          div,
                                          [a.cred[M.coinA][0] for a in M.agents],
                                          [a.cred[M.coinA][1] for a in M.agents],
                                          [a.cred[M.coinB][0] for a in M.agents],
                                          [a.cred[M.coinB][1] for a in M.agents],
                                          ['A' if c.bias==0.5 else 'B' for c in list(M.choices.values())]
                                          ]], 
                                        columns=cols), 
                           ignore_index=True)
    return(df)

In [27]:
def run_conformity():
    cols = ['k', 'w', 'epsilon', 'n_all', 'div', 
            'alphaA', 'betaA', 'alphaB', 'betaB', 'choices']
    df = pd.DataFrame(columns=cols)
    w = 1 # No intergroup distrust
    epsilon = 0.001 # difference between coin A and coin B biases
    n_all = 10 # agents per group
    for k in K: # levels of intergroup trust
        for div in [0.5, 0.7, 0.9]: # subgroup compositions 5-5, 7-3, 9-1
            M = Model(w=w, k=k, epsilon=epsilon, div=div, n_all=n_all)
            for t in range(3000): # 3000 time steps
                M.update()
                M.choose()
            df = df.append(pd.DataFrame([[k, 
                                          w,
                                          epsilon,
                                          n_all,
                                          div,
                                          [a.cred[M.coinA][0] for a in M.agents],
                                          [a.cred[M.coinA][1] for a in M.agents],
                                          [a.cred[M.coinB][0] for a in M.agents],
                                          [a.cred[M.coinB][1] for a in M.agents],
                                          ['A' if c.bias==0.5 else 'B' for c in list(M.choices.values())]
                                          ]], 
                                        columns=cols), 
                           ignore_index=True)
    return(df)

In [28]:
start = time.perf_counter()
run_conformity()
finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

Finished in 10.95 second(s)
